In [ ]:
# Directory 
input_path = os.environ.get('input_path')
fig_output = os.environ.get('fig_output')
tab_output = os.environ.get('tab_output')

# Queries config
project_id_bq = os.environ.get('project_id_bq')
run_query = os.environ.get('run_query')

In [ ]:
# Query region information from basedosdados 

def run_query_and_save_results():

    query = '''
    SELECT nome_regiao AS region, id_municipio AS municipality FROM basedosdados.br_bd_diretorios_brasil.municipio
    '''
    region = bd.read_sql(query, billing_project_id=project_id_bq)
    region.to_csv(os.path.join(input_path,'region.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

## Delay Payment Maps

In [ ]:
# Input Budget Execution Index, which has some statistics about budget execution

df = pd.read_csv(os.path.join(input_path, 'full_budget_execution_index.csv'))
df['wavg_delay'] = df['wavg_delay'].replace(np.nan, 1000).astype(float).replace(1000, "")
df = df.query('year == 2018 & wavg_delay != ""')
df['wavg_delay'] = df['wavg_delay'].astype(float)
df['municipality'] = df['municipality'].astype(str)

# Input region information from basedosdados municipality directory

region = pd.read_csv(os.path.join(input_path, 'region.csv'))
region['municipality'] = region['municipality'].astype(str)

# Geometry file and join with below dataframe
ufs = gpd.read_file(os.path.join(input_path, 'BRUFE250GC_SIR.shp'))
brazil = gpd.read_file(os.path.join(input_path, 'BRMUE250GC_SIR.shp'))
brazil['CD_GEOCUF'] = brazil['CD_GEOCMU'].str.slice(0, 2)
brazil = pd.merge(brazil, ufs, how='left', on=['CD_GEOCUF'])
brazil = pd.merge(brazil, region, how='left', left_on=['CD_GEOCMU'], right_on=['municipality'])
map = pd.merge(brazil, df, how='left', left_on=['CD_GEOCMU'], right_on=['municipality'])
map = GeoDataFrame(map)
# Restrict the area plot just to data. Yet, we don't have North, MiddleWest and some states from Northeast in database
map_northeast = map.query('region == "Nordeste"').drop('geometry_y', axis=1).rename(columns={'geometry_x':'geometry'})
map_south_southeast = map.query('region == "Sul" | region == "Sudeste"').drop('geometry_y', axis=1).rename(columns={'geometry_x':'geometry'})

In [ ]:
# Weighted Average Delay Payment
# Instead of using a continuous colormap, use discrete intervals for clarity
bins = [1, 12, 18, 24, 30, 38, 48, 70, 85, 100]
colors = ['#f0f9e8','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#08589e', '#1a476f', '#191970']
cmap = mpl.colors.ListedColormap(colors)
norm = mpl.colors.BoundaryNorm(bins, cmap.N)

# Creation of image with Weighted Average Delay Payment
fig, ((ax1, ax2)) = plt.subplots(2, 1, figsize=(8,10))
ax = plt.gca()
gs = gridspec.GridSpec(2, 1, height_ratios=[0.2, 0.6]) #better proportion betweens regions
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

# Plot southeast & south region
im1 = map_south_southeast.plot(column='wavg_delay', ax=ax2, legend=False, cmap=cmap, linewidth=0.3, norm=norm, zorder=1)
ufs.query('NM_REGIAO == "SUL" | NM_REGIAO == "SUDESTE"').plot(ax=ax2, color='none', edgecolor='black', linewidth=0.5, zorder=2)
ax2.axis('off')

# Plot some states from northeast region
im2 = map_northeast.plot(column='wavg_delay', ax=ax1, legend=False, cmap=cmap, linewidth=0.3, norm=norm, zorder=1)
ufs.query('NM_ESTADO == "CEARÁ" | NM_ESTADO == "RIO GRANDE DO NORTE" | NM_ESTADO == "PARAÍBA" | NM_ESTADO == "PERNAMBUCO"').plot(ax=ax1, color='none', edgecolor='black', linewidth=0.5, zorder=2)
ax1.axis('off')

# Adjust colorbar positioning and add it to the figure
cbar_ax_position = [0.2, 0.01, 0.4, 0.02]  # [left, bottom, width, height]
cbar_ax = fig.add_axes(cbar_ax_position)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax = cbar_ax, orientation='horizontal', ticks = bins, drawedges = False, boundaries = bins)
cbar.set_label('Weighted Average payment delay (days)', rotation=0, labelpad=10)
cbar.outline.set_visible(False)

for tick in cbar.ax.get_xticks():
    cbar.ax.axvline(tick, color='black', linewidth=0.6)

# Adjust the spacing between the subplots
fig.subplots_adjust(wspace=-0.5, bottom=0.02)  # Adjust the bottom space to make room for the colorbar

south_southeast_dissolved = ufs.query('NM_REGIAO == "SUL" | NM_REGIAO == "SUDESTE"').dissolve(by='NM_REGIAO')
northeast_dissolved = ufs.query('NM_ESTADO == "CEARÁ" | NM_ESTADO == "RIO GRANDE DO NORTE" | NM_ESTADO == "PARAÍBA" | NM_ESTADO == "PERNAMBUCO"').dissolve(by='NM_REGIAO')

# Plot only the external border of the dissolved shape
south_southeast_dissolved.plot(ax=ax2, color='none', edgecolor='black', linewidth = 0.1)
northeast_dissolved.plot(ax=ax1, color='none', edgecolor='black', linewidth = 0.1)

# Set titles with y parameter to align them
title_position = 1.02
ax2.set_title(r"Minas Gerais, São Paulo, Paraná"
              "\n" r"and Rio Grande do Sul", y=title_position)
ax1.set_title("Ceará and Paraíba", y=title_position)

plt.tight_layout()

plt.savefig(os.path.join(fig_output, 'wavg_delay_2018.pdf'), bbox_inches='tight')
plt.savefig(os.path.join(fig_output, 'wavg_delay_2018.png'), transparent=True, dpi=fig.dpi, bbox_inches='tight')

In [ ]:
# Share of commitments paid over 30 days by municipality
# Instead of using a continuous colormap, use discrete intervals for clarity
bins = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
colors = ['#f0f9e8','#ccebc5','#a8ddb5','#7bccc4','#4eb3d3','#2b8cbe','#08589e', '#1a476f', '#191970']
cmap = mpl.colors.ListedColormap(colors)
norm = mpl.colors.BoundaryNorm(bins, cmap.N)

# Creation of image with Over 30 days payment
fig, ((ax1, ax2)) = plt.subplots(2, 1, figsize=(8,10))
ax = plt.gca()
gs = gridspec.GridSpec(2, 1, height_ratios=[0.2, 0.6])
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

# Plot southeast & south region
im1 = map_south_southeast.plot(column='over_30days', ax=ax2, legend=False, cmap=cmap, linewidth=0.3, norm=norm, zorder=1)
ufs.query('NM_REGIAO == "SUL" | NM_REGIAO == "SUDESTE"').plot(ax=ax2, color='none', edgecolor='black', linewidth=0.5, zorder=2)
ax2.axis('off')

# Plot some states from northeast region
im2 = map_northeast.plot(column='over_30days', ax=ax1, legend=False, cmap=cmap, linewidth=0.3, norm=norm, zorder=1)
ufs.query('NM_ESTADO == "CEARÁ" | NM_ESTADO == "RIO GRANDE DO NORTE" | NM_ESTADO == "PARAÍBA" | NM_ESTADO == "PERNAMBUCO"').plot(ax=ax1, color='none', edgecolor='black', linewidth=0.5, zorder=2)
ax1.axis('off')

# Adjust colorbar positioning and add it to the figure
cbar_ax_position = [0.2, 0.01, 0.4, 0.02]  # [left, bottom, width, height]
cbar_ax = fig.add_axes(cbar_ax_position)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax = cbar_ax, orientation='horizontal', ticks = bins, drawedges = False, boundaries = bins)
cbar.set_label('Share of payments paid over 30 days (%)', rotation=0, labelpad=10)
cbar.outline.set_visible(False)

for tick in cbar.ax.get_xticks():
    cbar.ax.axvline(tick, color='black', linewidth=0.6)

# Adjust the spacing between the subplots
fig.subplots_adjust(wspace=-0.5, bottom=0.02)  # Adjust the bottom space to make room for the colorbar

south_southeast_dissolved = ufs.query('NM_REGIAO == "SUL" | NM_REGIAO == "SUDESTE"').dissolve(by='NM_REGIAO')
northeast_dissolved = ufs.query('NM_ESTADO == "CEARÁ" | NM_ESTADO == "RIO GRANDE DO NORTE" | NM_ESTADO == "PARAÍBA" | NM_ESTADO == "PERNAMBUCO"').dissolve(by='NM_REGIAO')

# Plot only the external border of the dissolved shape
south_southeast_dissolved.plot(ax=ax2, color='none', edgecolor='black', linewidth = 0.1)
northeast_dissolved.plot(ax=ax1, color='none', edgecolor='black', linewidth = 0.1)

# Set titles with y parameter to align them
title_position = 1.02  # Adjust this value if necessary to change vertical position
ax2.set_title(r"Minas Gerais, São Paulo, Paraná"
              "\n" r"and Rio Grande do Sul", y=title_position)
ax1.set_title("Ceará and Paraíba", y=title_position)

plt.tight_layout()

plt.savefig(os.path.join(fig_output, 'over30_delay_2018.pdf'), bbox_inches='tight')
plt.savefig(os.path.join(fig_output, 'over30_delay_2018.png'), transparent=True, dpi=fig.dpi, bbox_inches='tight')